In [1]:
import os 
import sys
import random
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import lognorm
from scipy.stats import norm 
from scipy import integrate
from scipy.stats import gmean

from scipy.stats import pearsonr
from scipy.special import rel_entr

import statsmodels.api as sm 
import statsmodels.formula.api as smf

### src; https://github.com/arkottke/pyrotd
import pyrotd
pd.set_option('display.max_rows', 100)
from scipy import stats

from efficiency_OLS import OLS
from Linear_model_fits import fit_against_IM_and_M, fit_against_IM_and_M_Rjb, fit_against_IM_and_Rjb
from Linear_model_fits import fit_IM_against_M, fit_IM_against_Rjb, fit_IM_against_M_and_Rjb
from extract_edp_data import extract_EDP_data
from sufficiency_kl_divergence import compute_KL_divergence, compile_kl_divergence, summarize_agg_kld
from sufficiency_kl_divergence import compile_kld_mvn_EDPs, summarize_agg_kld_mvn
%load_ext autoreload
%autoreload 2

/Users/laxmandahal/opt/miniconda3/lib/python3.8/site-packages/statsmodels/compat/pandas.py:61: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [2]:
with open('BuildingNames.txt', 'r') as f:
    BuildingList = f.read() 
BuildingList = BuildingList.split('\n')

baseDir = r'/Users/laxmandahal/Desktop/UCLA/Phd/Research/IM_study'

## time period of the buildings
# T = np.array([0.13, 0.12, 0.22, 0.22, 0.16, 0.15, 0.26, 0.25, 0.49, 0.49])
T = np.array([0.13, 0.12, 0.16, 0.15, 0.22, 0.22, 0.26, 0.25, 0.49, 0.49])

IM_list = ['SaT1', 'PGA', 'PGV', 'Sa_avg', 'CAV', 'SI', 'ASI', 'DSI','DS_5to75', 'DS_5to95' ]

numGM = 826
g = 980.665 ## converts GM record to cm/sec^2

def compute_RotDxx_EDP(edpX, edpZ, percentile = 50):
    angles = np.arange(0, 180, step=1)
    radians = np.radians(angles)
    coeffs = np.c_[np.cos(radians), np.sin(radians)]
    
    edp_stacked = np.vstack([edpX, edpZ])
    rotated_edp = np.dot(coeffs, edp_stacked)
    percentile_edp = np.percentile(rotated_edp, q = percentile, axis = 0, interpolation='linear')
    return percentile_edp

def correlation_from_covariance(covariance):
    v = np.sqrt(np.diag(covariance))
    outer_v = np.outer(v, v)
    correlation = covariance / outer_v
    correlation[covariance == 0] = 0
    return correlation

In [3]:
BuildingList

['s1_48x32_high',
 's1_48x32_veryhigh',
 's1_96x48_high',
 's1_96x48_veryhigh',
 's2_48x32_high',
 's2_48x32_veryhigh',
 's2_96x48_high',
 's2_96x48_veryhigh',
 's4_96x48_high',
 's4_96x48_veryhigh']

In [5]:
gminfo_rotD50 = pd.read_csv(r'data/Complete_GM_info_RotD50.csv')
gminfo_rotD50 = gminfo_rotD50.set_index('key_0')
gminfo_rotD50.index.names = ['GMID']

In [5]:
buildingIndex = 0
pairingID = 1
# IMs = 'SaT1'
IMs = 'Sa_avg'
numGM = 826

if pairingID == 1:
    start_index_multiplier = 0
    end_index_multiplier = 1
else:
    start_index_multiplier = 2
    end_index_multiplier = 3

if IMs == 'SaT1':
    IM = gminfo_rotD50['T_%s'%T[buildingIndex]].values
else:
    IM = gminfo_rotD50[IMs]
    
baseDir = r'/Users/laxmandahal/Desktop/UCLA/Phd/Research/IM_study'
dataDir = os.path.join(baseDir, *['Results', 'IM_study_826GMs', BuildingList[buildingIndex]])

os.chdir(dataDir)
sdr = pd.read_csv(os.path.join(dataDir, 'SDR.csv'), header = None)
pfa = pd.read_csv(os.path.join(dataDir, 'PFA.csv'), header = None)

#story index
i = 0
j = 0
sdrX = sdr[3+i].values[numGM * start_index_multiplier : numGM * end_index_multiplier]
sdrZ = sdr[3+i].values[numGM * end_index_multiplier : numGM * (end_index_multiplier + 1)]

pfaX = pfa[4+i].values[numGM * start_index_multiplier : numGM * end_index_multiplier]
pfaZ = pfa[4+i].values[numGM * end_index_multiplier : numGM * (end_index_multiplier + 1)]

sdr_rotd50 = compute_RotDxx_EDP(sdrX, sdrZ, percentile=50)
pfa_rotd50 = compute_RotDxx_EDP(pfaX, pfaZ, percentile=50)
X = sm.add_constant(np.log(IM))

sdrX_avg = gmean([sdr[3+j].values[0:numGM], sdr[3+j].values[numGM*2:numGM*3]])
pfaX_avg = gmean([pfa[3+j].values[0:numGM], pfa[3+j].values[numGM*2:numGM*3]])

sdrZ_avg = gmean([sdr[3+j].values[numGM:numGM*2], sdr[3+j].values[numGM*3:numGM*4]])
pfaZ_avg = gmean([pfa[3+j].values[numGM:numGM*2], pfa[3+j].values[numGM*3:numGM*4]])
# print(pfaZ_avg.shape)
#                 compute_RotDxx_EDP(sdrX, sdrZ, percentile=50)
#                 pfa_rotD50 = compute_RotDxx_EDP(pfaX, pfaZ, percentile=50)
ols_sdrX = OLS(sdrX, IM)
ols_sdrZ = OLS(sdrZ, IM)
ols_pfaX = OLS(pfaX/g, IM)
ols_pfaZ = OLS(pfaZ/g, IM)

ols_sdrX_M = fit_against_IM_and_M(sdrX, IM, gminfo_rotD50['Magnitude'].values)

In [6]:
print(stats.shapiro(ols_sdrX.result.get_prediction().predicted_mean),
      stats.shapiro(ols_sdrZ.result.get_prediction().predicted_mean),
     stats.shapiro(ols_pfaX.result.get_prediction().predicted_mean),
     stats.shapiro(ols_pfaZ.result.get_prediction().predicted_mean))

ShapiroResult(statistic=0.9972583651542664, pvalue=0.17930753529071808) ShapiroResult(statistic=0.9972571134567261, pvalue=0.17902125418186188) ShapiroResult(statistic=0.9972577691078186, pvalue=0.1791643500328064) ShapiroResult(statistic=0.9972569346427917, pvalue=0.1789734661579132)


In [7]:
print(stats.shapiro(ols_sdrX.get_residual()),
      stats.shapiro(ols_sdrZ.get_residual()),
     stats.shapiro(ols_pfaX.get_residual()),
     stats.shapiro(ols_pfaZ.get_residual()))

ShapiroResult(statistic=0.951960027217865, pvalue=9.081196928608596e-16) ShapiroResult(statistic=0.9556958079338074, pvalue=4.438398009987436e-15) ShapiroResult(statistic=0.9433161020278931, pvalue=3.141224662742778e-17) ShapiroResult(statistic=0.9323948621749878, pvalue=7.300968022019399e-19)


In [8]:
np.cov(ols_sdrX.get_residual(), ols_pfaX.get_residual())

array([[0.20809287, 0.18095727],
       [0.18095727, 0.16696389]])

In [9]:
correlation_from_covariance(np.cov(ols_sdrX.get_residual(), ols_pfaX.get_residual()))

array([[1.        , 0.97081429],
       [0.97081429, 1.        ]])

In [10]:
# def kl_divergence_mvn(m0, S0, m1, S1):
#     """
#     Kullback-Liebler divergence from Gaussian pm,pv to Gaussian qm,qv.
#     Also computes KL divergence from a single Gaussian pm,pv to a set
#     of Gaussians qm,qv.
    

#     From wikipedia
#     KL( (m0, S0) || (m1, S1))
#          = .5 * ( tr(S1^{-1} S0) + log |S1|/|S0| + 
#                   (m1 - m0)^T S1^{-1} (m1 - m0) - N )
#     """
    
#     # store inv diag covariance of S1 and diff between means
#     N = m0.shape[0]
#     iS1 = np.linalg.inv(S1)
#     diff = m1 - m0

#     # kl is made of three terms
#     tr_term   = np.trace(iS1 @ S0)
#     det_term  = np.log(np.linalg.det(S1)/np.linalg.det(S0)) #np.sum(np.log(S1)) - np.sum(np.log(S0))
#     quad_term = diff.T @ np.linalg.inv(S1) @ diff #np.sum( (diff*diff) * iS1, axis=1)
#     #print(tr_term,det_term,quad_term)
#     return .5 * (tr_term + det_term + quad_term - N) 

# mu1 = np.array([np.mean(ols_sdrX.result.get_prediction().predicted_mean),
#                 np.mean(ols_pfaX.result.get_prediction().predicted_mean)])
# cov1 = np.cov(ols_sdrX.result.get_prediction().predicted_mean,
#               ols_pfaX.result.get_prediction().predicted_mean, ddof=0)
# mu2 = np.array([np.mean(ols_sdrZ.result.get_prediction().predicted_mean),
#                 np.mean(ols_pfaZ.result.get_prediction().predicted_mean)])
# cov2 = np.cov(ols_sdrZ.result.get_prediction().predicted_mean,
#               ols_pfaZ.result.get_prediction().predicted_mean, ddof=0)
# kl_divergence_mvn(mu1, cov1, mu2, cov2)

In [ ]:
stats.shapiro(np.log(IM))

In [ ]:
stats.shapiro(np.log(gminfo_rotD50['Magnitude']))

In [ ]:
stats.shapiro(np.log(gminfo_rotD50['Distance_Rjb']))

In [ ]:
stats.shapiro(np.log(gminfo_rotD50['T_0.26']))

In [ ]:
stats.shapiro(np.log(gminfo_rotD50['SI']))

In [ ]:
ols_IM = fit_IM_against_M(IM,gminfo_rotD50['Magnitude'] )
stats.shapiro(ols_IM.get_prediction().predicted_mean)

In [ ]:
stats.shapiro(ols_IM.resid)

In [6]:
IM_list = ['SaT1', 'PGA', 'PGV', 'Sa_avg', 'CAV', 'SI', 'ASI', 'DSI','DS_5to75', 'DS_5to95' ]

b = compile_kl_divergence(BuildingList, 9, baseDir, IM_list, gminfo_rotD50, pairingID=1, numGM=826,
                          remove_collapse= False, KL_on_residual=True, use_predicted_IM=False)
b

IM,SaT1,PGA,PGV,Sa_avg,CAV,SI,ASI,DSI,DS_5to75,DS_5to95
"KL(sdrX|im, sdrX|im,M)",0.000240,8.957884e-04,0.000003,2.010379e-04,0.000154,1.274097e-09,1.454360e-03,3.767535e-09,0.008935,0.005766
"KL(sdrZ|im, sdrZ|im,M)",0.000352,1.378655e-03,0.000010,2.891918e-04,0.000406,1.776515e-08,2.154831e-03,1.257434e-08,0.012157,0.007187
"KL(pfaX|im, pfaX|im,M)",0.000245,1.434685e-08,0.002047,4.493098e-04,0.012305,1.006796e-03,3.451064e-05,1.927541e-04,0.006698,0.001838
"KL(pfaZ|im, pfaZ|im,M)",0.000748,1.109524e-05,0.003116,1.827201e-03,0.018499,1.390711e-03,7.155687e-10,2.771981e-04,0.006050,0.001561
"KL(sdrX|im, sdrX|im,R)",0.003053,2.695342e-03,0.007020,1.423155e-03,0.027518,4.376955e-03,3.527109e-03,3.914693e-02,0.073360,0.071446
"KL(sdrZ|im, sdrZ|im,R)",0.001262,8.201756e-04,0.003867,3.723781e-04,0.022447,2.561740e-03,1.487946e-03,3.358103e-02,0.058403,0.058421
"KL(pfaX|im, pfaX|im,R)",0.000007,7.929486e-06,0.003221,1.364389e-05,0.016683,1.909133e-03,2.114694e-05,3.227860e-02,0.030209,0.033993
"KL(pfaZ|im, pfaZ|im,R)",0.000218,5.376725e-06,0.005751,1.779006e-09,0.023621,4.332000e-03,1.561278e-06,4.478385e-02,0.036804,0.039317
"KL(sdrX|im, sdrX|im,M,R)",0.010703,1.413637e-02,0.009416,6.202251e-03,0.029781,6.533461e-03,1.907809e-02,5.830814e-02,0.178103,0.163209
"KL(sdrZ|im, sdrZ|im,M,R)",0.006559,8.922188e-03,0.004517,3.110932e-03,0.022719,3.604435e-03,1.422889e-02,4.885019e-02,0.169284,0.149857


In [7]:
np.round(b.iloc[:4].sum(), 5) * 1000

IM
SaT1         1.58
PGA          2.29
PGV          5.18
Sa_avg       2.77
CAV         31.36
SI           2.40
ASI          3.64
DSI          0.47
DS_5to75    33.84
DS_5to95    16.35
dtype: float64

In [8]:
np.round(b.iloc[4:8].sum(),5)*1000

IM
SaT1          4.54
PGA           3.53
PGV          19.86
Sa_avg        1.81
CAV          90.27
SI           13.18
ASI           5.04
DSI         149.79
DS_5to75    198.78
DS_5to95    203.18
dtype: float64

In [9]:
np.round(b.iloc[:8].sum(), 5)*1000

IM
SaT1          6.12
PGA           5.81
PGV          25.03
Sa_avg        4.58
CAV         121.63
SI           15.58
ASI           8.68
DSI         150.26
DS_5to75    232.62
DS_5to95    219.53
dtype: float64

In [10]:
from sufficiency_kl_divergence import summarize_agg_kld

sumdf = summarize_agg_kld(BuildingList, baseDir, IM_list, gminfo_rotD50, pairingID=1, numGM=826,
                          remove_collapse= False, KL_on_residual=True, use_predicted_IM=False,
                         mv_causal_params=False) *1000
# sumdf
sumdf.style.highlight_min(color = 'lightgreen', axis = 1)


IM,SaT1,PGA,PGV,Sa_avg,CAV,SI,ASI,DSI,DS_5to75,DS_5to95
s1_48x32_high,3.796808,10.854653,84.956272,34.461972,309.390715,61.882683,6.259627,275.778455,224.855942,234.830198
s1_48x32_veryhigh,2.444900,10.715058,86.568587,34.530174,318.638748,62.848007,5.812537,279.864865,234.086923,243.343709
s1_96x48_high,8.252542,9.434691,94.257285,30.213134,301.865410,72.086101,8.296317,298.142403,234.455843,245.140396
s1_96x48_veryhigh,11.932779,10.889965,102.981826,31.932379,314.244180,80.132089,11.035104,317.965838,252.247831,263.179682
s2_48x32_high,15.518060,7.844856,90.292559,24.966529,281.876648,70.057479,11.286101,294.205788,240.499084,250.160111
s2_48x32_veryhigh,25.865540,16.040829,106.969924,30.150379,294.034037,87.622143,21.398247,333.155700,279.925525,289.900448
s2_96x48_high,31.553610,14.178437,95.510985,23.614177,293.155844,74.529771,21.356955,318.730114,299.803398,304.960255
s2_96x48_veryhigh,11.685883,5.505290,56.415951,11.136711,207.082023,43.289406,7.227119,235.485871,251.593231,256.576305
s4_96x48_high,21.387974,14.881163,44.746195,12.778063,182.732500,31.350568,21.583929,211.099963,290.207861,279.375729
s4_96x48_veryhigh,6.124877,5.814376,25.034131,4.575920,121.632244,15.577354,8.681466,150.260379,232.616685,219.528897


In [11]:
# os.chdir(baseDir)
# sumdf.to_csv('%s/Codes/data/agg_kld_univariate.csv'%baseDir)

In [ ]:
sumdf = summarize_agg_kld(BuildingList, baseDir, IM_list, gminfo_rotD50, pairingID=1, numGM=826,
                          remove_collapse= False, KL_on_residual=True, use_predicted_IM=True,
                         mv_causal_params=True)
# sumdf
sumdf.style.highlight_min(color = 'lightgreen', axis = 1)

In [12]:
from sufficiency_kl_divergence import compile_kld_mvn_EDPs
b = compile_kld_mvn_EDPs(BuildingList, 9, baseDir, IM_list, gminfo_rotD50, pairingID=1, numGM=826,
                          remove_collapse= False, KL_on_residual=True, use_predicted_IM=False)
b

IM,SaT1,PGA,PGV,Sa_avg,CAV,SI,ASI,DSI,DS_5to75,DS_5to95
"KL(sdr+pfaX|im, sdr+pfaX|im,M)",0.000929,0.000930,0.002419,0.001129,0.012867,0.001310,0.001680,0.000523,0.010953,0.005822
"KL(sdr+pfaZ|im, sdr+pfaZ|im,M)",0.001956,0.001730,0.003629,0.002867,0.018882,0.001990,0.002162,0.000924,0.012922,0.007256
"KL(sdr+pfaX|im, sdr+pfaX|im,R)",0.003536,0.003983,0.010360,0.001609,0.046961,0.006984,0.005100,0.058211,0.079641,0.081129
"KL(sdr+pfaZ|im, sdr+pfaZ|im,R)",0.003107,0.001005,0.009321,0.000408,0.048215,0.006973,0.001581,0.059496,0.069300,0.071295
"KL(sdr+pfaX|im, sdr+pfaX|im,M,R)",0.011100,0.017889,0.012452,0.011495,0.053240,0.008547,0.020141,0.072134,0.206564,0.180889
"KL(sdr+pfaZ|im, sdr+pfaZ|im,M,R)",0.007691,0.011290,0.011075,0.006843,0.062035,0.008028,0.014346,0.068732,0.199707,0.165917


In [14]:
np.round(b.iloc[:2].sum(), 5) *1000

IM
SaT1         2.88
PGA          2.66
PGV          6.05
Sa_avg       4.00
CAV         31.75
SI           3.30
ASI          3.84
DSI          1.45
DS_5to75    23.88
DS_5to95    13.08
dtype: float64

In [15]:
np.round(b.iloc[2:4].sum(), 5)*1000

IM
SaT1          6.64
PGA           4.99
PGV          19.68
Sa_avg        2.02
CAV          95.18
SI           13.96
ASI           6.68
DSI         117.71
DS_5to75    148.94
DS_5to95    152.42
dtype: float64

In [16]:
np.round(b.iloc[:4].sum(), 5)*1000

IM
SaT1          9.53
PGA           7.65
PGV          25.73
Sa_avg        6.01
CAV         126.92
SI           17.26
ASI          10.52
DSI         119.15
DS_5to75    172.82
DS_5to95    165.50
dtype: float64

In [8]:
sumdf_mvn = summarize_agg_kld_mvn(BuildingList, baseDir, IM_list, gminfo_rotD50, pairingID=1, numGM=826,
                          remove_collapse= False, KL_on_residual=True, use_predicted_IM=False,
                         mv_causal_params=False)* 1000
# sumdf
sumdf_mvn.style.highlight_min(color = 'lightgreen', axis = 1) 

IM,SaT1,PGA,PGV,Sa_avg,CAV,SI,ASI,DSI,DS_5to75,DS_5to95
s1_48x32_high,6.345923,10.936014,61.956881,24.773305,182.598410,49.804758,8.138271,179.277891,141.350590,144.864758
s1_48x32_veryhigh,4.152443,9.748084,57.653308,23.037752,181.458788,45.314758,6.544981,169.383604,140.809399,144.283919
s1_96x48_high,8.694971,10.320115,66.118304,25.279704,185.760520,54.611728,9.883072,192.885804,147.244767,151.099665
s1_96x48_veryhigh,12.431618,12.770905,72.963744,28.930904,200.634803,60.756907,12.872700,207.533121,163.119619,166.708706
s2_48x32_high,18.373499,13.226619,76.752845,27.596492,209.110134,62.749438,16.720346,226.022613,186.198792,189.083077
s2_48x32_veryhigh,26.419409,19.783537,87.079957,33.398382,232.107040,73.155144,23.899011,246.506163,216.001332,218.580587
s2_96x48_high,32.195290,17.045196,79.766123,26.968761,234.230173,63.790840,22.288961,239.556154,230.590015,229.968679
s2_96x48_veryhigh,12.584535,7.388498,43.799459,11.503295,167.066814,34.167607,8.299429,159.924282,179.277034,180.364625
s4_96x48_high,27.816886,17.843887,47.066855,13.481860,190.590709,35.135560,25.834288,177.876041,237.773275,228.186667
s4_96x48_veryhigh,9.527809,7.647942,25.728490,6.013350,126.924616,17.256890,10.523485,119.154563,172.816699,165.502931


In [36]:
# sumdf_mvnir(baseDir)
# sumdf_mvn.to_csv('Codes/data/agg_kld_multivariate.csv')

In [10]:
sumdf_mvn = summarize_agg_kld_mvn(BuildingList, baseDir, IM_list, gminfo_rotD50, pairingID=1, numGM=826,
                          remove_collapse= False, KL_on_residual=True, use_predicted_IM=False,
                         mv_causal_params=True)
# sumdf
sumdf_mvn.style.highlight_min(color = 'lightgreen', axis = 1)

IM,SaT1,PGA,PGV,Sa_avg,CAV,SI,ASI,DSI,DS_5to75,DS_5to95
s1_48x32_high,0.006888,0.010766,0.061384,0.023866,0.211217,0.050297,0.009010,0.176835,0.186089,0.164998
s1_48x32_veryhigh,0.004781,0.009638,0.057316,0.022475,0.212204,0.045962,0.007459,0.167026,0.190791,0.167615
s1_96x48_high,0.008726,0.009912,0.063666,0.023573,0.206261,0.053481,0.009900,0.190514,0.192926,0.171470
s1_96x48_veryhigh,0.012433,0.012318,0.070564,0.026971,0.223139,0.059612,0.012917,0.204993,0.208819,0.187474
s2_48x32_high,0.018649,0.012708,0.072433,0.025438,0.216769,0.060838,0.016421,0.225840,0.258210,0.225657
s2_48x32_veryhigh,0.027172,0.019726,0.081567,0.031030,0.236859,0.070131,0.024154,0.246785,0.286972,0.258807
s2_96x48_high,0.033589,0.017635,0.074977,0.025027,0.234007,0.061401,0.023642,0.243337,0.360941,0.304545
s2_96x48_veryhigh,0.022709,0.015546,0.038905,0.014584,0.158395,0.031555,0.016928,0.169845,0.321724,0.280785
s4_96x48_high,0.036175,0.037093,0.043482,0.022150,0.176829,0.033466,0.050352,0.198823,0.499219,0.423206
s4_96x48_veryhigh,0.018791,0.029179,0.023526,0.018337,0.115275,0.016575,0.034487,0.140866,0.406271,0.346807


In [11]:
from suff_kld_prob_exceedance import compile_kld_prob_exceedance, summarize_kld_results

a = compile_kld_prob_exceedance(BuildingList, 9, baseDir, IM_list, gminfo_rotD50, pairingID=1, numGM=826,
                               remove_collapse=False, use_predicted_IM=False)
a.style.highlight_min(color = 'lightgreen', axis = 1)

IM,SaT1,PGA,PGV,Sa_avg,CAV,SI,ASI,DSI,DS_5to75,DS_5to95
"KL(sdrX|im, sdrX|im,M)",0.000002,0.000007,0.000000,0.000001,0.000001,0.000000,0.000011,0.000000,0.000093,0.000061
"KL(sdrZ|im, sdrZ|im,M)",0.000002,0.000009,0.000000,0.000002,0.000003,0.000000,0.000014,0.000000,0.000115,0.000070
"KL(pfaX|im, pfaX|im,M)",0.000001,0.000000,0.000040,0.000002,0.000208,0.000017,0.000000,0.000010,0.000244,0.000092
"KL(pfaZ|im, pfaZ|im,M)",0.000004,0.000000,0.000052,0.000007,0.000251,0.000022,0.000000,0.000013,0.000164,0.000058
"KL(sdrX|im, sdrX|im,R)",0.000020,0.000020,0.000051,0.000009,0.000191,0.000030,0.000026,0.000309,0.000620,0.000604
"KL(sdrZ|im, sdrZ|im,R)",0.000007,0.000005,0.000025,0.000002,0.000136,0.000016,0.000009,0.000252,0.000473,0.000475
"KL(pfaX|im, pfaX|im,R)",0.000000,0.000000,0.000062,0.000000,0.000273,0.000032,0.000000,0.001144,0.000894,0.001206
"KL(pfaZ|im, pfaZ|im,R)",0.000001,0.000000,0.000092,0.000000,0.000310,0.000066,0.000000,0.001334,0.000775,0.000999
"KL(sdrX|im, sdrX|im,M,R)",0.000065,0.000095,0.000067,0.000039,0.000205,0.000044,0.000125,0.000438,0.001277,0.001189
"KL(sdrZ|im, sdrZ|im,M,R)",0.000033,0.000052,0.000029,0.000016,0.000138,0.000022,0.000081,0.000351,0.001139,0.001038


In [16]:
a = summarize_kld_results(BuildingList, baseDir, IM_list, gminfo_rotD50, pairingID = 1,
                          numGM = 826, remove_collapse = False, KL_on_residual=True,
                         use_predicted_IM=False, mv_causal_params=False) * 100000
a.style.highlight_min(color = 'lightgreen', axis = 1)

IM,SaT1,PGA,PGV,Sa_avg,CAV,SI,ASI,DSI,DS_5to75,DS_5to95
s1_48x32_high,0.086828,1.874589,99.608597,10.273667,279.550222,68.414712,0.814499,608.496422,330.891361,440.271435
s1_48x32_veryhigh,0.089139,1.606159,77.065996,8.755599,219.652795,52.054207,0.715296,479.511442,269.800012,357.875386
s1_96x48_high,1.490910,2.980558,120.538649,13.173212,330.470716,83.168320,1.948424,714.446184,380.717581,510.574763
s1_96x48_veryhigh,1.853919,3.183807,114.872575,13.464646,316.352653,78.831779,2.279088,687.828525,360.541433,489.047481
s2_48x32_high,8.522292,4.314449,86.891714,14.008527,249.538872,60.315946,6.084479,554.680130,291.926388,399.200407
s2_48x32_veryhigh,12.047554,7.846328,114.435444,17.122760,308.644787,83.221996,10.013729,630.717644,343.093481,450.836606
s2_96x48_high,14.181336,6.616649,74.686261,11.919962,225.553148,53.366937,9.681353,414.448359,300.612893,366.442197
s2_96x48_veryhigh,6.361863,3.142721,70.987297,6.547827,226.518372,49.959944,3.864579,440.568145,331.811907,400.504406
s4_96x48_high,13.177474,10.005463,41.795574,7.584957,166.168158,26.535452,14.560834,312.397353,345.507027,361.558138
s4_96x48_veryhigh,3.633336,4.031607,32.165188,2.382682,137.394750,18.358319,5.943450,306.227550,337.659299,356.561145


In [17]:
a = summarize_kld_results(BuildingList, baseDir, IM_list, gminfo_rotD50, pairingID = 1,
                          numGM = 826, remove_collapse = False, KL_on_residual=True,
                         use_predicted_IM=False, mv_causal_params=True) * 100000
a.style.highlight_min(color = 'lightgreen', axis = 1)

IM,SaT1,PGA,PGV,Sa_avg,CAV,SI,ASI,DSI,DS_5to75,DS_5to95
s1_48x32_high,0.148561,1.874761,92.409639,10.221201,273.188813,66.992605,0.921500,592.086328,429.998695,504.702949
s1_48x32_veryhigh,0.187958,1.607368,71.442275,8.742096,215.213961,50.972012,0.858184,466.645489,352.839978,412.806520
s1_96x48_high,1.495784,3.333973,111.678180,13.528222,320.800181,81.369818,2.294512,693.870239,491.652614,582.259612
s1_96x48_veryhigh,1.856894,3.577440,106.522291,13.883915,307.978222,77.234795,2.615648,667.536136,465.849892,557.047963
s2_48x32_high,8.621279,5.515375,80.290941,16.240919,241.596330,58.867918,7.167925,538.943814,384.945261,460.590425
s2_48x32_veryhigh,12.265647,8.499740,106.400347,18.614736,299.626889,81.181291,10.446835,613.961069,446.828969,517.381429
s2_96x48_high,14.953033,6.740071,70.247071,12.902061,222.797821,52.246908,10.079593,408.146711,426.577356,452.094142
s2_96x48_veryhigh,11.805080,4.825932,66.270560,6.956583,219.714853,48.898617,7.064717,441.429175,490.758607,516.840289
s4_96x48_high,20.297809,21.086016,40.314395,12.181271,163.280639,26.848640,30.798129,328.649442,566.360560,542.393777
s4_96x48_veryhigh,10.563217,14.701556,32.441291,6.903599,135.738378,20.083338,20.668049,330.311593,576.268841,556.356862


$$ P(EDP>edp | IM) = 1 - \Phi\left( \frac{\ln(edp) - \hat\mu_{\ln EDP|IM}}{\sigma_{\ln EDP|IM}} \right)$$


$$ P(EDP>edp | IM,M) = 1 - \Phi\left( \frac{\ln(edp) - \hat\mu_{\ln EDP|IM,M}}{\sigma_{\ln EDP|IM,M}} \right)$$


$$ P(EDP>edp | IM,R) = 1 - \Phi\left( \frac{\ln(edp) - \hat\mu_{\ln EDP|IM,R}}{\sigma_{\ln EDP|IM,R}} \right)$$


$$ P(EDP>edp | IM,M,R) = 1 - \Phi\left( \frac{\ln(edp) - \hat\mu_{\ln EDP|IM,M,R}}{\sigma_{\ln EDP|IM,M,R}} \right)$$